In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/oily-mlgo-hackathon-data/sample_submission.csv
/kaggle/input/oily-mlgo-hackathon-data/train.csv
/kaggle/input/oily-mlgo-hackathon-data/test.csv


importing the train data

In [2]:
train_data=pd.read_csv('/kaggle/input/oily-mlgo-hackathon-data/train.csv')
train_data.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS
0,-999.0,-999.0,41.4699,-999.0,-999.0,-999.0,-999.0,128.0737,319.0654
1,-999.0,-999.0,42.5053,-999.0,-999.0,-999.0,-999.0,127.8347,318.7825
2,-999.0,-999.0,43.1548,-999.0,-999.0,-999.0,-999.0,127.2307,317.3323
3,-999.0,-999.0,43.2410,-999.0,-999.0,-999.0,-999.0,126.2917,313.6486
4,-999.0,-999.0,40.3218,-999.0,-999.0,-999.0,-999.0,125.3985,307.8903


importing test data

In [3]:
test_data=pd.read_csv('/kaggle/input/oily-mlgo-hackathon-data/test.csv')
test_data.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN
0,8.5781,0.3521,55.1824,0.8121,0.7810,6.8291,2.3256
1,8.5781,0.3639,57.0114,0.8038,0.7723,6.8100,2.3255
2,8.5781,0.3703,58.9263,0.7444,0.7048,6.7766,2.3212
3,8.5625,0.3667,57.3308,0.7169,0.6542,6.7219,2.3119
4,8.5781,0.3500,53.0624,0.6845,0.6109,6.6384,2.2982


figuring out the important features for prediction of the target variables

In [4]:
X_train=train_data.iloc[:,0:7]
X_train.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN
0,-999.0,-999.0,41.4699,-999.0,-999.0,-999.0,-999.0
1,-999.0,-999.0,42.5053,-999.0,-999.0,-999.0,-999.0
2,-999.0,-999.0,43.1548,-999.0,-999.0,-999.0,-999.0
3,-999.0,-999.0,43.2410,-999.0,-999.0,-999.0,-999.0
4,-999.0,-999.0,40.3218,-999.0,-999.0,-999.0,-999.0


In [5]:
y1_train=train_data['DTC']
y2_train=train_data['DTS']
y1_train.head()

0    128.0737
1    127.8347
2    127.2307
3    126.2917
4    125.3985
Name: DTC, dtype: float64

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30143 entries, 0 to 30142
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CAL     30143 non-null  float64
 1   CNC     30143 non-null  float64
 2   GR      30143 non-null  float64
 3   HRD     30143 non-null  float64
 4   HRM     30143 non-null  float64
 5   PE      30143 non-null  float64
 6   ZDEN    30143 non-null  float64
dtypes: float64(7)
memory usage: 1.6 MB


check for null values

In [7]:
X_train.isnull().sum()

CAL     0
CNC     0
GR      0
HRD     0
HRM     0
PE      0
ZDEN    0
dtype: int64

In [8]:
from sklearn.feature_selection import mutual_info_regression
mi=mutual_info_regression(X_train,y1_train)
mi_scores=pd.Series(mi,name="MI Scores",index=X_train.columns)
mi_scores

CAL     1.063369
CNC     1.176628
GR      0.688087
HRD     0.886722
HRM     0.814098
PE      0.492043
ZDEN    0.729638
Name: MI Scores, dtype: float64

using mutual information to rank the features according to target importance

In [9]:
features=mi_scores[mi_scores>0.7].index.tolist()

In [10]:
print(features)

['CAL', 'CNC', 'HRD', 'HRM', 'ZDEN']


In [11]:
X_train.shape

(30143, 7)

In [12]:
y1_train.shape

(30143,)

splitting of datasets

In [13]:
from sklearn.model_selection import train_test_split
X1_Train,X1_test,y1_Train,y1_test=train_test_split(X_train,y1_train,test_size=0.2)
X2_Train,X2_test,y2_Train,y2_test=train_test_split(X_train,y2_train,test_size=0.2)

In [14]:
X1_Train.shape

(24114, 7)

In [15]:
y1_Train.shape

(24114,)

fitting and predicting the first target variable

In [16]:
from sklearn.ensemble import RandomForestRegressor
model1=RandomForestRegressor()
model1.fit(X1_Train,y1_Train)
pred1=model1.predict(X1_test)

fitting and predicting the second target variable

In [17]:
model2=RandomForestRegressor()
model2.fit(X2_Train,y2_Train)
pred2=model2.predict(X2_test)

In [18]:
from sklearn.metrics import mean_squared_error
rmse1= np.sqrt(mean_squared_error(y1_test,pred1))
rmse2= np.sqrt(mean_squared_error(y2_test,pred2))

In [19]:
print(rmse1)

50.303409816959544


In [20]:
print(rmse2)

131.2545865174295


priyanka sucks